# Cost Optimization & Resource Efficiency

## Overview
This notebook implements cost optimization and resource efficiency strategies. It analyzes resource usage, identifies optimization opportunities, and tracks cost savings.

## Prerequisites
- Completed: `security-incident-response-automation.ipynb`
- Resource usage metrics available
- Cost tracking data available
- Billing information accessible

## Learning Objectives
- Analyze resource utilization
- Identify cost optimization opportunities
- Implement efficiency improvements
- Track cost savings
- Optimize resource allocation

## Key Concepts
- **Resource Utilization**: Measure actual vs allocated
- **Cost Analysis**: Track spending patterns
- **Optimization**: Reduce waste and improve efficiency
- **Right-Sizing**: Match resources to actual needs
- **ROI**: Calculate return on optimization investments

## Setup Section

In [ ]:
import sys
import os
import json
import logging
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from typing import Dict, List, Any

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR = DATA_DIR / 'reports'
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Configuration
NAMESPACE = 'self-healing-platform'
CPU_COST_PER_HOUR = 0.05  # $ per CPU hour
MEMORY_COST_PER_GB_HOUR = 0.01  # $ per GB hour

logger.info(f"Cost optimization initialized")

## Implementation Section

### 1. Analyze Resource Utilization

In [ ]:
def analyze_utilization(resource_data: pd.DataFrame) -> Dict[str, Any]:
    """
    Analyze resource utilization patterns.
    
    Args:
        resource_data: Resource usage data
    
    Returns:
        Utilization analysis
    """
    try:
        analysis = {
            'timestamp': datetime.now().isoformat(),
            'cpu_analysis': {
                'allocated': resource_data['cpu_allocated'].sum(),
                'used': resource_data['cpu_used'].sum(),
                'utilization_rate': (resource_data['cpu_used'].sum() / resource_data['cpu_allocated'].sum()),
                'wasted': resource_data['cpu_allocated'].sum() - resource_data['cpu_used'].sum()
            },
            'memory_analysis': {
                'allocated': resource_data['memory_allocated'].sum(),
                'used': resource_data['memory_used'].sum(),
                'utilization_rate': (resource_data['memory_used'].sum() / resource_data['memory_allocated'].sum()),
                'wasted': resource_data['memory_allocated'].sum() - resource_data['memory_used'].sum()
            }
        }
        
        logger.info(f"CPU utilization: {analysis['cpu_analysis']['utilization_rate']:.1%}")
        logger.info(f"Memory utilization: {analysis['memory_analysis']['utilization_rate']:.1%}")
        return analysis
    except Exception as e:
        logger.error(f"Utilization analysis error: {e}")
        return {'error': str(e)}

# Create resource data
resource_data = pd.DataFrame([
    {
        'pod': f'pod-{i}',
        'cpu_allocated': np.random.uniform(0.5, 2),
        'cpu_used': np.random.uniform(0.1, 1.5),
        'memory_allocated': np.random.uniform(256, 1024),
        'memory_used': np.random.uniform(100, 800)
    }
    for i in range(20)
])

utilization = analyze_utilization(resource_data)
print(json.dumps(utilization, indent=2, default=str))

### 2. Calculate Cost Metrics

In [ ]:
def calculate_costs(resource_data: pd.DataFrame, hours: int = 24) -> Dict[str, Any]:
    """
    Calculate resource costs.
    
    Args:
        resource_data: Resource usage data
        hours: Time period in hours
    
    Returns:
        Cost metrics
    """
    try:
        total_cpu_allocated = resource_data['cpu_allocated'].sum()
        total_cpu_used = resource_data['cpu_used'].sum()
        total_memory_allocated = resource_data['memory_allocated'].sum()
        total_memory_used = resource_data['memory_used'].sum()
        
        # Calculate costs
        allocated_cpu_cost = total_cpu_allocated * CPU_COST_PER_HOUR * hours
        used_cpu_cost = total_cpu_used * CPU_COST_PER_HOUR * hours
        allocated_memory_cost = (total_memory_allocated / 1024) * MEMORY_COST_PER_GB_HOUR * hours
        used_memory_cost = (total_memory_used / 1024) * MEMORY_COST_PER_GB_HOUR * hours
        
        costs = {
            'timestamp': datetime.now().isoformat(),
            'period_hours': hours,
            'allocated_cost': allocated_cpu_cost + allocated_memory_cost,
            'actual_cost': used_cpu_cost + used_memory_cost,
            'wasted_cost': (allocated_cpu_cost - used_cpu_cost) + (allocated_memory_cost - used_memory_cost),
            'cost_efficiency': (used_cpu_cost + used_memory_cost) / (allocated_cpu_cost + allocated_memory_cost)
        }
        
        logger.info(f"Wasted cost: ${costs['wasted_cost']:.2f}")
        return costs
    except Exception as e:
        logger.error(f"Cost calculation error: {e}")
        return {'error': str(e)}

costs = calculate_costs(resource_data, hours=24)
print(json.dumps(costs, indent=2, default=str))

### 3. Identify Optimization Opportunities

In [ ]:
def identify_optimizations(utilization: Dict, costs: Dict) -> Dict[str, Any]:
    """
    Identify cost optimization opportunities.
    
    Args:
        utilization: Utilization analysis
        costs: Cost metrics
    
    Returns:
        Optimization opportunities
    """
    try:
        opportunities = {
            'timestamp': datetime.now().isoformat(),
            'recommendations': [],
            'potential_savings': 0
        }
        
        # CPU optimization
        cpu_util = utilization['cpu_analysis']['utilization_rate']
        if cpu_util < 0.5:
            savings = costs['wasted_cost'] * 0.5
            opportunities['recommendations'].append({
                'type': 'cpu_right_sizing',
                'description': f'CPU utilization is {cpu_util:.1%}, consider reducing allocations',
                'potential_savings': savings
            })
            opportunities['potential_savings'] += savings
        
        # Memory optimization
        mem_util = utilization['memory_analysis']['utilization_rate']
        if mem_util < 0.5:
            savings = costs['wasted_cost'] * 0.3
            opportunities['recommendations'].append({
                'type': 'memory_right_sizing',
                'description': f'Memory utilization is {mem_util:.1%}, consider reducing allocations',
                'potential_savings': savings
            })
            opportunities['potential_savings'] += savings
        
        logger.info(f"Identified {len(opportunities['recommendations'])} optimization opportunities")
        return opportunities
    except Exception as e:
        logger.error(f"Optimization identification error: {e}")
        return {'error': str(e)}

optimizations = identify_optimizations(utilization, costs)
print(json.dumps(optimizations, indent=2, default=str))

### 4. Track Cost Savings

In [ ]:
# Create cost tracking dataframe
cost_tracking = pd.DataFrame([
    {
        'timestamp': datetime.now() - timedelta(days=i),
        'allocated_cost': np.random.uniform(100, 500),
        'actual_cost': np.random.uniform(50, 400),
        'wasted_cost': np.random.uniform(10, 150),
        'cpu_utilization': np.random.uniform(0.3, 0.8),
        'memory_utilization': np.random.uniform(0.4, 0.85),
        'optimizations_applied': np.random.randint(0, 5)
    }
    for i in range(30)  # 30 days of data
])

# Save tracking data
tracking_file = PROCESSED_DIR / 'cost_optimization_tracking.parquet'
cost_tracking.to_parquet(tracking_file)

# Generate report
report = {
    'report_date': datetime.now().isoformat(),
    'period_days': 30,
    'total_allocated_cost': cost_tracking['allocated_cost'].sum(),
    'total_actual_cost': cost_tracking['actual_cost'].sum(),
    'total_wasted_cost': cost_tracking['wasted_cost'].sum(),
    'cost_efficiency': cost_tracking['actual_cost'].sum() / cost_tracking['allocated_cost'].sum(),
    'avg_cpu_utilization': cost_tracking['cpu_utilization'].mean(),
    'avg_memory_utilization': cost_tracking['memory_utilization'].mean()
}

report_file = REPORTS_DIR / f"cost_optimization_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_file, 'w') as f:
    json.dump(report, f, indent=2, default=str)

logger.info(f"Saved cost tracking data and report")
print(cost_tracking.to_string())

## Validation Section

In [ ]:
# Verify outputs
assert tracking_file.exists(), "Cost tracking file not created"
assert report_file.exists(), "Report file not created"

total_savings = cost_tracking['wasted_cost'].sum()
avg_efficiency = cost_tracking['actual_cost'].sum() / cost_tracking['allocated_cost'].sum()

logger.info(f"✅ All validations passed")
print(f"\nCost Optimization & Resource Efficiency Summary:")
print(f"  Tracking Records: {len(cost_tracking)}")
print(f"  Total Wasted Cost (30 days): ${total_savings:.2f}")
print(f"  Cost Efficiency: {avg_efficiency:.1%}")
print(f"  Average CPU Utilization: {report['avg_cpu_utilization']:.1%}")
print(f"  Average Memory Utilization: {report['avg_memory_utilization']:.1%}")
print(f"  Optimization Opportunities: {len(optimizations.get('recommendations', []))}")

## Integration Section

This notebook integrates with:
- **Input**: Resource usage and cost data
- **Output**: Optimization recommendations and cost reports
- **Monitoring**: Cost tracking and efficiency metrics
- **Next**: Complete notebook roadmap

## Summary

This completes the **Advanced Scenarios Phase** with 4 comprehensive notebooks:

1. ✅ Multi-Cluster Healing Coordination
2. ✅ Predictive Scaling & Capacity Planning
3. ✅ Security Incident Response Automation
4. ✅ Cost Optimization & Resource Efficiency

## Final Steps

1. Deploy all advanced scenarios
2. Monitor platform performance
3. Collect feedback and iterate
4. Optimize based on real-world usage
5. Celebrate 30/30 notebooks complete! 🎉

## References

- ADR-003: Self-Healing Platform Architecture
- ADR-012: Notebook Architecture for End-to-End Workflows
- [FinOps Framework](https://www.finops.org/)
- [Kubernetes Cost Optimization](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/)